In [1]:
import numpy as np
import glob
from PIL import Image
from skimage import morphology
from tifffile import imread, imwrite
from tqdm import tqdm
import napari

In [2]:
def save_stack(imgx, imgy, imgz, d_type = None, return_stacks=True):
    imgx_stack = []
    imgy_stack = []
    imgz_stack = []

    for j in tqdm(range(len(imgx))):
        ix = Image.open(imgx[j])
        iy = Image.open(imgy[j])
        iz = Image.open(imgz[j])


        ix.load()
        iy.load()
        iz.load()

        #imgc = np.array(imgc, dtype='uint8')
        #imgn = np.array(imgn, dtype='uint8')
        #imgp = np.array(imgp, dtype='uint8')
        if d_type == None:
            ix = np.array(ix)
            iy = np.array(iy)
            iz = np.array(iz)
        else:
            ix = np.array(ix, dtype=d_type)
            iy = np.array(iy, dtype=d_type)
            iz = np.array(iz, dtype=d_type)

        imgx_stack.append(ix)
        imgy_stack.append(iy)
        imgz_stack.append(iz)

    imgx_stack = np.array(imgx_stack)
    imgy_stack = np.array(imgy_stack)
    imgz_stack = np.array(imgz_stack)
        
    if return_stacks == True:
        return imgx_stack, imgy_stack, imgz_stack

In [ ]:
## rerun=True if running for first time
## rerun=False simply loads the new tiffs from file

## Save/Load data from file

In [5]:
rerun = False

if rerun == True:
    chro = []
    for file in glob.glob("chromosome_1800_2802/*.jpg"):
        chro.append(file)

    nucl = []
    for file in glob.glob("nucleolus_1800_2802/*.jpg"):
        nucl.append(file)

    patt = []
    for file in glob.glob("pattern_1800_2803/*.jpg"):
        patt.append(file)
        
    n_imgs = 999

    chro = sorted(chro[0:n_imgs])
    nucl = sorted(nucl[0:n_imgs])
    patt = sorted(patt[0:n_imgs])
    chro, nucl, patt= save_stack(chro, nucl, patt)

    imwrite('chromosome_1800_2802.tiff', chro)
    imwrite('nucleolus_1800_2802.tiff', nucl)
    imwrite('pattern_1800_2802.tiff', patt)
else:
    chro = imread('chromosome_1800_2802.tiff')
    nucl = imread('nucleolus_1800_2802.tiff')
    patt = imread('pattern_1800_2802.tiff')


## Remove nucleolus false positives

We use the morphology.remove_small_objects module from sklearn

Parameter to toggle is min_size. Would be interesting if this affects watershedding of chromosomes.
I settle on min_size values of :
- 9500 for nucleolus
- 800 for chromosomes
- 300 for patterns

In [7]:
def clean_stack(img_stack, minim):
    cleaned = np.copy(img_stack)
    cleaned_index = (cleaned!=0)

    for j in tqdm(range(len(cleaned))):
        img = cleaned_index[j,:] 
        img = morphology.remove_small_objects(img, minim, connectivity=1)
        target_img = cleaned[j,:,:]

        cleaned[j,:,:] = np.multiply(target_img, img)
        
    return cleaned

In [8]:
if rerun == True:
    nucl_cleaned = clean_stack(nucl, min_size=9500)
    chro_cleaned = clean_stack(chro, min_size=800)
    patt_cleaned = clean_stack(patt, min_size=300)
    
    imwrite('chromosome_1800_2802_cleaned.tiff', chro_cleaned)
    imwrite('nucleolus_1800_2802_cleaned.tiff', nucl_cleaned)
    imwrite('pattern_1800_2802_cleaned.tiff', patt_cleaned)
else:    
    nucl_cleaned = imread('nucleolus_1800_2802_cleaned.tiff')
    chro_cleaned = imread('chromosome_1800_2802_cleaned.tiff')
    patt_cleaned = imread('pattern_1800_2802_cleaned.tiff')



## Make movie

In [14]:
from napari_animation import AnimationWidget

viewer = napari.Viewer()
viewer.add_image(patt_cleaned, name='pattern', colormap='cyan')
viewer.add_image(chro_cleaned, name='chromosome', colormap='red')
viewer.add_image(nucl_cleaned, name='nucleolus')

animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')

## Make GIF

In [20]:
gif_dir = 'not_aligned'
def make_gif(frame_folder, name):
    frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*.png")]
    frame_one = frames[0]
    frame_one.save(name + '.gif', format="GIF", append_images=frames,
               save_all=True, duration=100, loop=0)
    
make_gif(gif_dir, 'my')